__This contains the master controls and dashboards for all tools. No development should be done herein__

In [ ]:
%cd ~/Dropbox/CanvasHacks

import pandas as pd
pd.options.display.max_rows = 999

from CanvasHacks.Widgets.DashboardControls import button_area, make_control_store
control_store = make_control_store()
button_area(control_store)

# SKAA
## Students who haven't turned in the Essay / content assignment

In [ ]:
control_store['skaa_dash'].no_essay

## Students whose reviewer hasn't turned in the review

In [ ]:
control_store['skaa_dash'].non_reviewed

## Reviewers whose author hasn't turned in metareview

Here we needd to look at the reviewing field. The 'student' is the person who didn't turn in the metareview

In [ ]:
control_store['skaa_dash'].non_metareviewed

# Discussions
## Students who haven't posted to the forum

In [ ]:
control_store['diss_dash'].non_posters

## Students whose reviewers didn't do the review

In [ ]:
control_store['diss_dash'].non_reviewed

In [ ]:
control_store['diss_dash'].reviewed